In [17]:
# from distributed import Client
# client = Client()
import pandas as pd
# import modin.pandas as pd
import os
import json
from pathlib import Path
from urllib.request import urlretrieve
from tqdm import tqdm
from zipfile import ZipFile
from google.cloud import bigquery
from prefect import task, flow
from prefect_gcp.cloud_storage import GcsBucket
pd.set_option("display.max_columns", None)
print("Setup Complete")

Setup Complete


In [18]:
# Seq 1-Define a function to convert the downloaded file to data frame
def read_df(file: str) -> pd.DataFrame:
    with open(file) as data_file:
        data = json.load(data_file)
        df = pd.read_json(data)
        df = pd.json_normalize(df.to_dict("records"), sep="_")
        return df
    
# file = "/Users/reneboygarcia/Library/CloudStorage/GoogleDrive-reneboygarcia@gmail.com/My Drive/Personal/Data Science Notebook/Data Engineering-Zoomcamp/week_7_capstone_project/albums-json/albums-full-info-11.json"
# file_10 = "/Users/reneboygarcia/Library/CloudStorage/GoogleDrive-reneboygarcia@gmail.com/My Drive/Personal/Data Science Notebook/Data Engineering-Zoomcamp/week_7_capstone_project/albums-json/albums-full-info-10.json"
# file_9 = "/Users/reneboygarcia/Library/CloudStorage/GoogleDrive-reneboygarcia@gmail.com/My Drive/Personal/Data Science Notebook/Data Engineering-Zoomcamp/week_7_capstone_project/albums-json/albums-full-info-9.json"
# df = read_df(file_9)
# df.head()

In [19]:
# df.info(memory_usage="deep")

In [20]:
# Seq 2-Define a function to tweak the data frame
def tweak_df(df: pd.DataFrame) -> pd.DataFrame:
    print(f"Number of rows: {df.shape[0]}")
    df["datePublished"] = pd.to_datetime(df["datePublished"], errors="coerce")
    df["dateModified"] = pd.to_datetime(df["dateModified"], errors="coerce")
    df_ = df.drop(
        columns=["albumRelease", "@type", "image", "@id", "@context", "@graph", "inAlbum", "recordingOf", "offers", "track"],
        errors="ignore",
    )
    return df_

df_tweak = tweak_df(df)
df_tweak.head()

Number of rows: 100000


,_id,description,url,name,duration_secs,keywords,dateModified,datePublished,duration,numTracks,isrcCode,comment,offers_@type,offers_url,offers_priceSpecification_minPrice,offers_availability,offers_priceCurrency,offers_price,byArtist_genre,byArtist_@type,byArtist_name,byArtist_@id,byArtist_sameAs,byArtist_image,byArtist_description,track_numberOfItems,track_itemListElement,track_@type,inAlbum_@type,inAlbum_name,inAlbum_@id,recordingOf_@type,recordingOf_lyrics_text,recordingOf_lyrics_@type
0,https://lilahlarson.bandcamp.com/track/tbh,Words and music by Lilah Larson\r\nProduced by...,https://lilahlarson.bandcamp.com/track/tbh,tbh,267.981,"Rock, americana, folk, genre queer, indie",2016-10-12 17:23:25+00:00,2016-10-12 16:39:22+00:00,P00H04M27S,NaN,NaN,NaN,Offer,https://lilahlarson.bandcamp.com/track/tbh#buy,0.0,OnlineOnly,USD,0.0,https://bandcamp.com/tag/rock,MusicGroup,Lilah Larson,https://lilahlarson.bandcamp.com,[http://www.lilahlarson.com],https://f4.bcbits.com/img/0008491492_10.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://versusny.bandcamp.com/album/hurrah,NaN,NaN,Hurrah,NaN,"Alternative, New York",2017-03-09 17:45:48+00:00,2016-02-01 19:27:24+00:00,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://bandcamp.com/tag/alternative,MusicGroup,Versus,https://versusny.bandcamp.com,NaN,https://f4.bcbits.com/img/0015735441_10.jpg,"Versus was formed in 1990 by Richard Baluyut, ...",12.0,"[{'item': {'name': 'My Adidas', 'url': 'https:...",ItemList,NaN,NaN,NaN,NaN,NaN,NaN
2,https://helllight.bandcamp.com/album/no-god-ab...,"The fourth full-length album from HellLight, o...",NaN,"No God Above, No Devil Below",NaN,"Doom Death Metal, Metal, death doom, doom meta...",2019-09-24 08:27:12+00:00,2013-07-14 21:20:37+00:00,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://bandcamp.com/tag/metal,MusicGroup,HellLight,https://helllight.bandcamp.com,"[https://www.facebook.com/helllightdoom/, http...",https://f4.bcbits.com/img/0010486672_10.jpg,NaN,8.0,"[{'item': {'name': 'Intro', 'duration': 'P00H0...",ItemList,NaN,NaN,NaN,NaN,NaN,NaN
3,https://hyponic.bandcamp.com/album/-,NaN,NaN,前行者,NaN,"Doom Metal, Funeral Doom Metal, Metal, Psyched...",2016-08-09 15:59:52+00:00,2016-07-02 16:22:17+00:00,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://bandcamp.com/tag/metal,MusicGroup,HYPONIC,https://hyponic.bandcamp.com,"[http://hyponic.wix.com/hyponic, https://www.f...",https://f4.bcbits.com/img/0007841378_10.jpg,HYPONIC is a Band from Hong Kong formed in 199...,6.0,"[{'@type': 'ListItem', 'item': {'@type': ['Mus...",ItemList,NaN,NaN,NaN,NaN,NaN,NaN
4,https://spiraltrax.bandcamp.com/track/breaking...,NaN,https://spiraltrax.bandcamp.com/track/breaking...,Breaking Limits,549.933,"Electronic, psychedelic trance, psytrance, tec...",2015-09-17 21:10:14+00:00,2015-09-17 21:12:45+00:00,P00H09M09S,NaN,NaN,NaN,Offer,https://spiraltrax.bandcamp.com/track/breaking...,1.0,OnlineOnly,USD,1.0,https://bandcamp.com/tag/electronic,MusicGroup,Human Blue,https://spiraltrax.bandcamp.com,NaN,https://f4.bcbits.com/img/0004655611_10.jpg,NaN,NaN,NaN,NaN,MusicAlbum,Human Blue - Ice,https://spiraltrax.bandcamp.com/album/human-bl...,NaN,NaN,NaN


In [21]:
# df_tweak.info(memory_usage="deep")

In [22]:
# directory = Path("bandcamp")
# file_name = file_10.split("/")[-1].split(".")[0]
# path_name = directory / f"{file_name}.parquet"
# path_name

In [23]:
# @flow(retries=2, retry_delay_seconds=10)
# def main(df, path_name):
#     gcs_block = GcsBucket.load("prefect-gcs-block-bandcamp")
#     gcs_block.upload_from_dataframe(df=df, to_path=path_name, serialization_format="parquet")
#     return

# main(df_tweak,path_name)

In [24]:
# Seq 3-Define a function to upload dataframe to GCS
# https://github.com/PrefectHQ/prefect-gcp/pull/140
def write_local(df: pd.DataFrame, filename: str) -> Path:
    directory = Path("bandcamp")
    _file_name = filename.split("/")[-1].split(".")[0]
    path_name = directory / f"{_file_name}.parquet"
    try:
        os.makedirs(directory, exist_ok=True)
        # directory.mkdir()
        print("Converting json file to parquet....")
        df.to_parquet(path_name, compression="snappy")
        print("Done converting to parquet....")
    except OSError as error:
        print(error)
    return path_name

# write_local(df_tweak, filename_path=file_10)


In [25]:
# Seq 4-Define a function to upload local file to GCS Bucket
@flow()
def write_to_gcs(path: Path) -> None:
    gcs_block = GcsBucket.load("prefect-gcs-block-bandcamp")
    gcs_block.upload_from_path(from_path=path, to_path=path)
    print("Hooray, we uploaded a huge file in GCS")
    return

# path_json = write_local(df_tweak, file_9)
# write_to_gcs(path_json)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py:214: UserWarning: A flow named 'write-to-gcs' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_19585/2220231410.py:2' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [26]:
# Seq 5-Delete local file and its directory
def duduplicate(path: Path) -> None:
    try:
        path.unlink()
        full_path = path.resolve()
        full_path.parent.rmdir()
        print("Successfully deleted directory and its files")
    except OSError as error:
        print(f"Unable to find directory: {error}")

In [27]:
# Define ETL from web to gcs:
def etl_web_to_gcs(file: str):
    # Seq 1 -Read file
    df = read_df(file)
    # Seq 2 -Tweak df
    df_ = tweak_df(df)
    # Seq 3 -Set a path this will be use to convert file to parquet
    path_file = write_local(df_, file)
    # Seq 4-Upload local file to GCS Bucket
    write_to_gcs(path_file)
   

In [28]:
# Define download progress hook
def download_progress_hook(block_num, block_size, total_size):
    global progress_bar
    if not progress_bar:
        progress_bar = tqdm(total=total_size, unit='B', unit_scale=True)
    downloaded = block_num * block_size
    progress_bar.update(downloaded - progress_bar.n)
    if downloaded >= total_size:
        progress_bar = None

In [29]:
# Seq 0 -Download file folder from web
def fetch_data(url:str):
    folder_name = url.split("/")[-1].split("?")[0]
    file_folder = urlretrieve(url, folder_name, reporthook=download_progress_hook)
    if folder_name.endswith(".zip"):
        zip_file = ZipFile(folder_name)
        folder_name_ = os.path.commonprefix(zip_file.namelist()).strip("/")
        zip_file.extractall()
        print(f"Download Complete..extracted zip file")
        print(f"Extracted folder path: {folder_name_}")
        return folder_name_
    print(f"Download Complete..")
    return file_folder

In [30]:
# Define a parent ETL to download the files
def elt_parent_web_gcs():
    # Parameters
    dataset_url = "https://www.dropbox.com/s/a1kl5e35j4o53mz/bandcamp-items-json.zip?dl=1"

    # Execution
    # Seq 0 -Download file folder from web
    file_folder = fetch_data(dataset_url)
    # Loop through the files then run etl_web_to_gcs
    print("Running etl_web_to_gcs...this will take sometime..grab some coffee or tea")
    for file in os.listdir(file_folder)[2:3]:
        if file.endswith(".json"):
            file_path = os.path.join(file_folder, file)
            print(f"Running: {file}")
            etl_web_to_gcs(file_path)
            print(f"Done uploading {file} to GCS")
    
    # Seq 5- Remove duplicate
    duduplicate(file_path)
    print("All files are Uploaded")

In [31]:
# # dataset_url = "https://www.dropbox.com/s/a1kl5e35j4o53mz/bandcamp-items-json.zip?dl=1"
# dataset_url_2 = "https://www.dropbox.com/s/wd38q80el16i19q/1000000-bandcamp-sales.zip?dl=1"
# file_folder = fetch_data(dataset_url_2)
# for file in os.listdir(file_folder):
#     if file.endswith(".json"):
#         print(file)

In [32]:
if __name__ == "__main__":
    progress_bar = None
    elt_parent_web_gcs()

# Note: It seems download_progress_hook is not working when transferred to python file. :( 
# Deleted Seq 5 and place the option to cache the fetch data on the python file
# I will retain this on this notebook to automatically delete the folder and files on local. 


988MB [06:32, 2.52MB/s]                            


Download Complete..extracted zip file
Extracted folder path: albums-json
Running etl_web_to_gcs...this will take sometime..grab some coffee or tea
Running: albums-full-info-5.json
Number of rows: 100000
Converting json file to parquet....
Done converting to parquet....


10:03:42.618 | INFO    | prefect.engine - Created flow run 'dazzling-earwig' for flow 'write-to-gcs'

10:03:45.859 | INFO    | Flow run 'dazzling-earwig' - Getting bucket 'prefect-gcs-bucket-bandcamp'.

10:03:46.288 | INFO    | Flow run 'dazzling-earwig' - Uploading from PosixPath('bandcamp/albums-full-info-5.parquet') to the bucket 'prefect-gcs-bucket-bandcamp' path 'bandcamp/albums-full-info-5.parquet'.

10:04:46.429 | ERROR   | Flow run 'dazzling-earwig' - Encountered exception during execution:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/engine.py", line 665, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*args, **kwargs)
  File "/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_19585/2220231410.py", line 5, in write_to_gcs
    gcs_block.upload_from_path(from_path=path, to_path=path)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/events/instrument.py", line 73, in inner
    raise exc
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/events/instrument.py", line 70, in inner
    return function(self, *args, **kwargs)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 226, in coroutine_wrapper
    return run_async_from_worker_thread(async_fn, *args, **kwargs)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 177, in run_async_from_worker_thread
    return anyio.from_thread.run(call)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/from_thread.py", line 49, in run
    return asynclib.run_async_from_thread(func, *args)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 970, in run_async_from_thread
    return f.result()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/concurrent/futures/_base.py", line 446, in result
    return self.__get_result()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/concurrent/futures/_base.py", line 391, in __get_result
    raise self._exception
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect_gcp/cloud_storage.py", line 1135, in upload_from_path
    await run_sync_in_worker_thread(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 91, in run_sync_in_worker_thread
    return await anyio.to_thread.run_sync(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/google/cloud/storage/blob.py", line 2683, in upload_from_filename
    self.upload_from_file(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/google/cloud/storage/blob.py", line 2539, in upload_from_file
    created_json = self._do_uploa

10:04:46.923 | ERROR   | Flow run 'dazzling-earwig' - Finished in state Failed("Flow run encountered an exception. requests.exceptions.ConnectionError: ('Connection aborted.', timeout('The write operation timed out'))\n")

ConnectionError: ('Connection aborted.', timeout('The write operation timed out'))